# **Workshop VII** <br/> *Bayesian inference and decision trees*

In this exercise, please fill in the blanks that indicated by ...

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import beta, norm, bernoulli

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from decimal import Decimal

import ID3  # decision tree module

---

## **1.** Bayesian Basics

This first exercise is meant as an introduction to the Bayesian framework. In particular, we will simulate a dataset of disease contamination. By simulating the dataset, we know the "true" parameter values and we can thus examine how accurate our inferences are. To get started, lets set up the simulation parameters.

In [ ]:
# simulate a population of 100,000 people:
population = 100_000
prevalence = 0.01   # disease prevalence
sensitivity = 0.95  # true positive rate
specificity = 0.90  # true negative rate

# infectious status (1 = infected, 0 = not infected):
population_status = np.random.choice([1, 0], size=population, p=[prevalence, 1-prevalence])

# assign some fictitious test results to the population, based on our sensitivity and specificity:
test_results = np.zeros(population)
for i in range(population):
    if population_status[i] == 1:
        test_results[i] = np.random.choice([1, 0], p=[sensitivity, 1-sensitivity])
    else:
        test_results[i] = np.random.choice([1, 0], p=[1-specificity, specificity])


In [ ]:
# create a dataframe to store the population data:
df = pd.DataFrame({'status': population_status, 'test_result': test_results})

#### Now, assuming that we have access to both the test results and the true results (the "status"), try to extract the Baysian probabilities from our simulated data, to find out posterior probability of being infected, given a positive test result. 

In [ ]:
# calculate the Bayesian probabilities:
prior_probability = ...
likelihood = ...
probability_testing_positive = ...

print('Prior probability of infection:', prior_probability)
print('Likelihood of testing positive given infection:', likelihood)
print('Probability of testing positive:', probability_testing_positive)

#### Complete the function to calculate posterior, and them use it to get the posterior.

In [ ]:
# Bayesian inference of Pr(status=1 | test=1)
def bayes_theorem(prior, likelihood, evidence):
    return ...

pr_infected_given_positive = ...

# print posterior. which is (Pr(test=1 | status=1) * Pr(status=1)) / Pr(test=1)
print('Pr(status=1 | test=1) = ', pr_infected_given_positive) 

#### Interpret your finding.

### If sensitivity is increased to 0.999, the specificity remains as original, what is the value of Pr(status=1 | test=1)?

#### If specificity is increased to 0.999, the sensitivity remains as original, what is the value of Pr(status=1 | test=1)?

#### Interprete your finding.

---

## **2.** Dealing with Uncertainty

Of course, we are never able to take a peak into the true population parameters in real life. There will always be uncertainty. So far you have tried to quantify this uncertainty using frequentist techniques (i.e. maximum likelihood estimations, etc). Today you will quantify your uncertainty differently, using Bayesian statistics.

In the following, you will try to use Bayesian inference to go from data to results.

Now suppose we want to estimate the average weight of fish in a fishery farm. The real average weight of the fish is 2 kg. but we do not know this value beforehand. After a number of catches, we get data on the weight of a batch of fish. We know that the weights of the detected fish obeys a normal distribution around the mean value. How can we obtain the actual average weight of the fish from this data?

Let's start by adding some small data that obeys normal distribution. The mean is set to be 2 and the standard deviation is 1.

The Data generated from the following code:

#np.random.seed = 39

#sample_data = np.random.normal(2, 1, 30)

In [ ]:
sample_data = np.array([ 1.64350291,  0.87752135,  3.05569158,  1.48274427,  1.4059861 ,
        3.93554187,  0.53890017,  2.12058382,  0.54360397,  2.25308647,
        2.71833633,  2.11059019,  0.90982909,  3.10162428,  2.99375448,
        1.9048221 ,  2.25047556,  2.60237373,  1.59678977,  2.00217239,
        1.3469326 ,  1.05712171,  0.32794892,  3.36094034, -0.42828078,
        1.27685994,  2.84693071,  3.21779923,  0.15080283,  2.29198599])

Now, because we cannot obtain the true value anymore, we set up a *prior distribution* to represent our uncertainty. 

Set up 4 priors to compare their effect on the posterior distribution later. Use the population value obtained in the previous part (i.e. approximately 0.0.1).

In [ ]:
x_range = np.linspace(1,3,100)

# Prior 1: uncertain
prior1 = beta.pdf(x_range/3.0, 1, 1)/3.0

# Prior 2: somewhat uncertain
prior2 = beta.pdf(x_range/3.0, 1, 3)/3.0

# Prior 3: somewhat certain
prior3 = beta.pdf(x_range/3.0, 5, 3)/3.0

# Prior 4: certain
prior4 = beta.pdf(x_range/3.0, 30, 15)/3.0

In [ ]:
for prior, title in zip([prior1, prior2, prior3, prior4], ['Uncertain', 'Somewhat uncertain', 'Somewhat certain', 'Certain']):
    plt.plot(x_range, prior, label=title)

plt.xlabel('Probability of infection')
plt.ylabel('Density')
plt.title('Priors')
plt.legend()

#### Equiped with the prior distributions, we now form the likelihood.

In [ ]:
# multiply log elements together
def multiply_elements(array):
    return np.log(array).sum()

# computing exponential function and normalization for large value
def largeExpNormalize(x_range,loglikelihoodlist):
    declikelihood = [Decimal(k).exp() for k in loglikelihoodlist]
    
    surface = Decimal(0)
    for i in range(1,len(x_range)):
        surface += Decimal(x_range[i]-x_range[i-1])*declikelihood[i]        
    
    mlh = [float(k/surface) for k in declikelihood]
    return mlh

# Gaussian pdf
def loglikelihood_function_normal(data,miu,sigma):
    return multiply_elements(norm.pdf(data, miu, sigma))

# compute likelihood
mlh = ...

In [ ]:
plt.plot(x_range, mlh)
plt.scatter(x_range[np.argmax(mlh)], mlh[np.argmax(mlh)], color='red',label="max %.4f"%(x_range[np.argmax(mlh)]))
plt.legend()

#### Compute the posterior with prior 1-4 and plot them. A function to compute the area under the curve is available.

In [ ]:
def getSurface(x,y):
    surface = 0
    for i in range(1,len(x)):
        surface += (x[i]-x[i-1])*y[i]        
    return surface 

In [ ]:
posterior = ...
# plot...

Interpret the above plots.

---

## 3. Decision Trees: ID3

In this section, you will use the provided `ID3.py` algorithm implementation to build and interpret a decision tree classifier. Apply this classifier to a real-world dataset, perform some predictions, and then analyze the decision tree's structure.

Information on the variables in this dataset:
1. age: age in years
2. sex: sex (1 = male; 0 = female)
3. cp: chest pain type:
    - Value 1: typical angina
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic
4. trestbps: resting blood pressure (in mm Hg on admission to the hospital)
5. chol: serum cholestoral in mg/dl
6. fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
7. restecg: resting electrocardiographic results
    - Value 0: normal
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
8. thalach: maximum heart rate achieved   
9. exang: exercise induced angina (1 = yes; 0 = no)
10. oldpeak = ST depression induced by exercise relative to rest
11. slope: the slope of the peak exercise ST segment
    - Value 1: upsloping
    - Value 2: flat
    - Value 3: downsloping
12. ca: number of major vessels (0-3) colored by flourosopy        
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. num: diagnosis of heart disease (angiographic disease status)
    - Value 0: < 50% diameter narrowing, which is no disease.
    - Value more than 0: > 50% diameter narrowing (in any major vessel), which indicates disease.

Load the "Heart Disease UCI" dataset & remove missing values.

In [ ]:
# import data & add column names
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'
]
df = pd.read_csv('processed.cleveland.data', names=column_names, header=None)
df.head()

In [ ]:
# drop missing values
...

Convert the continuous attributes into categorical ones. We need to do for attributes age, trestbps, chol, thalach, oldpeak, cp, sec, slope, thal and num. The first two are already done.

Note that there are many ways to do the discretization.

In [ ]:
# Binning 'age' into categories
bins_age = [0, 30, 40, 50, 60, 70, 120]
labels_age = ['below 30', '30-39', '40-49', '50-59', '60-69', 'above 70']
df['age'] = pd.cut(df['age'], bins=bins_age, labels=labels_age, include_lowest=True)

# Binning 'trestbps' (resting blood pressure) into categories
bins_trestbps = [0, 120, 140, 160, 180, 200]
labels_trestbps = ['below 120', '121-140', '141-160', '161-180', 'above 180']
df['trestbps'] = pd.cut(df['trestbps'], bins=bins_trestbps, labels=labels_trestbps, include_lowest=True)

# Binning 'chol' (serum cholesterol) into categories
...

# Binning 'thalach' (maximum heart rate achieved) into categories
...

# Binning 'oldpeak' (ST depression induced by exercise relative to rest) into categories
...

# Giving cp (chest pain type) more descriptive names
...

# Giving sex more descriptive names
...

# Giving slope (the slope of the peak exercise ST segment) more descriptive names
...

# Giving thal (thalium stress test result) more descriptive names
...

# Giving num (diagnosis of heart disease) more descriptive names
...

# Turn the rest into string
df = df.astype(str)

# show
print([type(df.iloc[0,i]) for i in range(len(df.columns))])
df.head()

#### Split the dataset into training and test sets.

In [ ]:
# Split the dataset into training and test sets
data_train, data_test = ...

#### Now, using the provided ID3 algorithm, construct a decision tree using the training set.

In [ ]:
# Construct the decision tree from the provided ID3 algorithm
decision_tree = ...

In [ ]:
# Print the tree to get an idea of its structure
ID3.print_tree(decision_tree)

#### Implement a code to use the constructed decision tree to make predictions on the test set.

In [ ]:
# The 'class' column should not be included in the features of the test set
test_features = ...
actual_values = ...

# Use the new prediction function to get predictions for the test set
# default value is used when no result is obtained
predictions = ...

# Compare the predictions with the actual values
comparison = pd.DataFrame({'Actual': actual_values, 'Predicted': predictions})

print(comparison)

In [ ]:
# Lets visualize the accuracy!
accuracy = accuracy_score(actual_values, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Generate the confusion matrix
conf_matrix = confusion_matrix(actual_values, predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=['Predicted No', 'Predicted Yes'], yticklabels=['Actual No', 'Actual Yes'])
plt.title('Confusion Matrix')
#plt.xlabel('Predicted Label')
#plt.ylabel('True Label')
plt.show()

Finally, interpret your decision tree:

In [ ]:
ID3.print_tree(decision_tree)

---

## **4.** Decision Trees: CART

This time we try to use the python build-in function, CART.

For this method, we don't need to categorize our attributes. While the non-categorical attributes works better.

In [ ]:
# Reload the data
df = pd.read_csv('processed.cleveland.data', names=column_names, header=None)
df.replace('?', np.nan, inplace=True)  # missing entries are marked with '?'
df.dropna(inplace=True)

X = df.drop('num', axis=1)  # features
y = df['num']  # target

# Split the dataset into training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print([type(X_train.iloc[0,i]) for i in range(len(X_train.columns))])

#### Create a decision tree and fit the data into it.

In [ ]:
# Create a DecisionTreeClassifier object
cart_model = ...

# Fit the model to the training data
...

#### Now let's make some predictions on the test set to evaluate performance:

In [ ]:
# Making predictions
predictions = ...

# Calculate and print the accuracy
accuracy = ...
print(f"Accuracy: {accuracy:.4f}")

# Generate and plot the confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2, 3, 4], yticklabels=[0, 1, 2, 3, 4])
plt.title('Confusion Matrix')
plt.show()

Finally, we visualize the decision tree:

In [ ]:
plt.figure(figsize=(20,10))
plot_tree(cart_model, 
          filled=True, 
          rounded=True, 
          class_names=['0', '1', '2', '3', '4'],  # Update class names as appropriate
          feature_names=list(X_train.columns))       # Ensure X_train.columns is accessible
plt.show()

#### Please interpret the results, try to answer the following:

- How well did both models predict heart disease presence?
- Describe the model's performance in terms of false positives, false negatives, true positives, and true negatives.
- Analyze the tree to determine the most important features and decision paths leading to heart disease prediction.